This notebook is a handy tool for converting CNNs produced by `train.py` to fully convolutional CNNs for evaluation.

In [ ]:
%matplotlib inline

from keras.models import model_from_json
from keras.optimizers import SGD

from os import path

from train import infer_sizes
import models

Configuration and metadata (layer size) gathering

In [ ]:
cache_dir = '../cache/mpii-cooking/' # Change me!
orig_path = path.join(cache_dir,  'keras-checkpoints/checkpoints/model-iter-16640-r349513.h5') # Change me!
train_h5_path = path.join(cache_dir, 'train-patches/samples-000001.h5')
ds_shape = infer_sizes(train_h5_path)

Load a model

In [ ]:
sgd2 = SGD(lr=0.0001, nesterov=True, momentum=0.9)
poselet_model = models.vggnet16_poselet_class_flow(ds_shape, sgd2, 'glorot_normal')
poselet_model.load_weights(orig_path)

Upgrade the model

In [ ]:
upgraded_poselet_model = models.upgrade_multipath_poselet_vggnet(poselet_model)
assert poselet_model.loss, "Model needs a non-empty loss"
upgraded_poselet_model.compile(sgd2, poselet_model.loss)

Save a description of the model and its new weights to `cnn_model.{json,h5}` in the cache directory.

In [ ]:
mod_json = upgraded_poselet_model.to_json()
mod_json_path = path.join(cache_dir, 'cnn_model.json')
mod_weight_path = path.join(cache_dir, 'cnn_model.h5')
with open(mod_json_path, 'w') as fp:
    fp.write(mod_json)
upgraded_poselet_model.save_weights(mod_weight_path)

(Optional) Re-read the model to verify that it wrote out correctly.

In [ ]:
def ask(question):
    res = raw_input(question + ' (y[es] for affirmative, any other response for negative) ')
    return res.lower() in ['y', 'yes']

In [ ]:
if ask('Want to re-read the model?'):
    with open(mod_json_path) as fp:
        json_data = fp.read()
    m2 = model_from_json(json_data)
    m2.load_weights(mod_weight_path)

(Optional) visualise the models

In [ ]:
if ask('Want to visualise the model?'):
    SVG(to_graph(poselet_model, show_shape=True).create(prog='dot', format='svg'))